### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

# Imports

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

# Initialize and Constants

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found.")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key.")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them.")
else:
    print("API key found and looks good so far!")


MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key found and looks good so far!


# Scrape Website

In [28]:
class Website:
    url: str
    title: str
    body: str
    links: List[str]
    text: str
    
    def __init__(self, url):
        self.url = url
        reponse = requests.get(url)
        self.body = reponse.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
    
    def get_content(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
"One shot prompting" will be used in which an example of how it should respond in the prompt is provided.

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [10]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [19]:
aiub = Website("https://www.aiub.edu/")
aiub.links
print(get_links_user_prompt(aiub))

Here is the list of links on the website of https://www.aiub.edu/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
http://www.enable-javascript.com/
#
#
#
#
//portal.aiub.edu/
//portal.office.com/
/contact-us
/about
/about/Information
/about/general-information
/about/Rules-Of-Entry
/about/Why-Study-Here
/about/Resources
/about/career
/about/Convocation
/about/Video
#
/faculties/fass
/faculties/fba
/faculties/engg
/faculties/b-pharm
/faculties/fst
/academic-calendar
/academic-regulations
#
/tuition-fee
/Faculty-List
/partnerships/internationalization
/partnerships/academic-partners
/partnerships/strategic-partners
https://ice.aiub.edu/
/iqac
/cetl
/admission
/
/onCampus
#
#
/Administration/The-Vice-Chancellor
/Administration/vice-chancellor-profile
/faculty-list/faculty-profile#arahman@aiub.edu
/Administratio

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages= [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [20]:
get_links("https://www.aiub.edu/")

{'links': [{'type': 'about page', 'url': 'https://www.aiub.edu/about'},
  {'type': 'careers page', 'url': 'https://www.aiub.edu/about/career'},
  {'type': 'admission page', 'url': 'https://admission.aiub.edu/'},
  {'type': 'scholarships page',
   'url': 'https://www.aiub.edu/admission/scholarships'},
  {'type': 'academic calendar',
   'url': 'https://www.aiub.edu/academic-calendar'},
  {'type': 'academic regulations',
   'url': 'https://www.aiub.edu/academic-regulations'},
  {'type': 'contact page', 'url': 'https://www.aiub.edu/contact-us'},
  {'type': 'future students page',
   'url': 'https://www.aiub.edu/future-Students'}]}

# Second Step: Make the Brochure

In [30]:
def get_all_details(url):
    result = "Landing Page:\n"
    result += Website(url).get_content()
    links = get_links(url)
    print("Found Links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_content()
    return result

In [ ]:
print(get_all_details("https://huggingface.co")) 

In [57]:
create_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website\
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information"

In [34]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [35]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding Page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nagentica-org/DeepCoder-14B-Preview\nUpdated\n4 days ago\n•\n6.87k\n•\n427\nmeta-llama/Llama-4-Scout-17B-16E-Instruct\nUpdated\n4 days ago\n•\n541k\n•\n757\nHiDream-ai/HiDream-I1-Full\nUpdated\nabout 13 hours ago\n•\n6.54k\n•\n333\nmoonshotai/Kimi-VL-A3B-Thinking\nUpdated\nabout 10 hours ago\n•\n4.69k\n•\n241\nnvidia/Llama-3_1-Nemotron-Ultra-253B-v1\nUpdated\n3 days ago\n•\n10.6k\n•\n213\nBrows

In [73]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": create_system_prompt},
            {
                "role": "user",
                "content": get_brochure_user_prompt(company_name, url),
            },
        ],
    )
    result = response.choices[0].message.content
    return result

In [75]:
display(Markdown(create_brochure("HuggingFace", "https://huggingface.co")))

Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

## About Us
Hugging Face is the AI community building the future, providing a collaborative platform for machine learning enthusiasts, researchers, and enterprises. With a mission to foster innovation and collaboration, we are the home of machine learning, supporting the creation, discovery, and deployment of advanced models and datasets.

- **Website:** [huggingface.co](https://huggingface.co)
- **Twitter:** [@huggingface](https://twitter.com/huggingface)
- **LinkedIn:** [Hugging Face](https://www.linkedin.com/company/huggingface)

## Our Offerings
Hugging Face offers a wide range of tools and resources to the AI community:
- **Models:** Over 1 million models to choose from, including popular ones like Llama-4 and DeepCoder.
- **Datasets:** Access to 250,000+ datasets for various machine learning tasks.
- **Spaces:** An interactive platform to create and showcase ML applications, featuring popular tools such as DeepSeek and EasyControl models.
- **Enterprise Solutions:** Offering paid compute options and tailored services for organizations with advanced AI needs.

## Customers
More than 50,000 organizations trust Hugging Face for their machine learning needs, from enterprise giants to non-profits. Our esteemed clientele includes:

- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

## Careers at Hugging Face
Join our rapidly growing team of over 200 dedicated professionals! We are always on the lookout for talented individuals who share our passion for artificial intelligence, collaboration, and innovation. Explore career opportunities across various functions, including engineering, research, and community engagement.

### Why Work With Us?
- **Innovative Culture:** We thrive on diversity, creativity, and a commitment to open-source principles. Our team embraces collaboration and continuous learning to build cutting-edge AI technologies.
- **Supportive Environment:** We encourage the sharing of ideas and knowledge within our community, fostering a collaborative atmosphere that empowers individual growth and team success.
- **Impactful Work:** Be part of a company that is shaping the future of artificial intelligence and machine learning, making a tangible impact across various industries.

## Get Involved
Whether you’re a researcher, developer, or simply passionate about AI, there’s a place for you in our community. Sign up today to start collaborating with like-minded individuals, explore our vast repository of models and datasets, or join discussions on the latest advancements in AI.

- **Explore Models:** [Explore 1M+ models](https://huggingface.co/models)
- **Join Our Community:** [Sign Up](https://huggingface.co/join)
- **Stay Updated:** Follow us on [Discord](https://discord.gg/huggingface) and [Twitter](https://twitter.com/huggingface) for the latest news and updates.

## Conclusion
At Hugging Face, we are committed to building an inclusive and innovative AI community. Join us as we shape the future of artificial intelligence together!

In [ ]:
display(Markdown(create_brochure("AIUB", "https://aiub.edu")))

Found Links: {'links': [{'type': 'about page', 'url': 'https://www.aiub.edu/about'}, {'type': 'career page', 'url': 'https://www.aiub.edu/about/career'}, {'type': 'contact page', 'url': 'https://www.aiub.edu/contact-us'}, {'type': 'admission page', 'url': 'https://www.aiub.edu/admission'}, {'type': 'tuition fee page', 'url': 'https://www.aiub.edu/tuition-fee'}, {'type': 'alumni page', 'url': 'https://www.aiub.edu/Alumni'}, {'type': 'academic calendar page', 'url': 'https://www.aiub.edu/academic-calendar'}, {'type': 'academic regulations page', 'url': 'https://www.aiub.edu/academic-regulations'}, {'type': 'faculties page', 'url': 'https://www.aiub.edu/faculties'}, {'type': 'research page', 'url': 'https://www.aiub.edu/research'}, {'type': 'sustainability page', 'url': 'https://www.aiub.edu/sustainability'}]}


# AIUB Brochure

### Welcome to American International University-Bangladesh (AIUB)

At AIUB, we believe in fostering the leaders of tomorrow through rigorous academic programs and an enriching campus culture. With a commitment to excellence and innovation, we prepare students for challenges in an ever-evolving world.

## Why Choose AIUB?

- **Diverse Academic Programs**: Our extensive range of undergraduate and postgraduate programs ensures there is something for everyone, including fields such as Business Administration, Engineering, Health and Life Sciences, and Arts and Social Sciences.

- **Technologically Advanced Education**: Our tech-centric teaching approach empowers students with the skills needed for today's digital age.

- **Global Partnerships**: AIUB maintains academic and strategic partnerships with various international institutions, enhancing the learning experience through global collaboration.

- **Sustainability Focus**: We are dedicated to responsible futures, emphasizing sustainability within our curriculum and campus initiatives.

## Campus Life and Culture

AIUB is not just about academic excellence; it’s about building a thriving community. Our urban green campus offers an inviting environment for students to learn, grow, and connect. We emphasize cultural affairs and sports, providing numerous opportunities for extracurricular engagement.

### Core Values
- **Innovation**: Encouraging creativity and new ideas.
- **Collaboration**: Building strong relationships within the campus community and beyond.
- **Integrity**: Upholding the highest standards of excellence and ethics.

## Career Opportunities

We value our talented faculty and staff, continuously seeking passionate individuals who wish to contribute to our vision. AIUB offers a dynamic work environment and numerous career advancement opportunities. Our Human Resources Office is dedicated to fostering professional growth and a positive workplace culture.

### Current Openings
Check our official website for the latest job vacancies and how to apply. Whether you're an educator, administrator, or support staff, AIUB welcomes diverse talent to join our community.

## Connect with Us

For prospective students, investors, and recruits looking for more information, feel free to explore our resources and reach out with your inquiries:

- [Visit our Website](www.aiub.edu)
- [Contact Us](mailto:info@aiub.edu)
- Follow us on [Social Media](#)

Join us at AIUB, where educational excellence meets innovation and opportunity. Together, we'll shape the future!

# With Changed Tone in System Prompt

In [56]:
create_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [ ]:
display(Markdown(create_brochure("AIUB", "https://aiub.edu")))

Found Links: {'links': [{'type': 'about page', 'url': 'https://www.aiub.edu/about'}, {'type': 'general information page', 'url': 'https://www.aiub.edu/about/general-information'}, {'type': 'careers page', 'url': 'https://www.aiub.edu/about/career'}, {'type': 'academic calendar', 'url': 'https://www.aiub.edu/academic-calendar'}, {'type': 'administration page', 'url': 'https://www.aiub.edu/Administration'}, {'type': 'contact page', 'url': 'https://www.aiub.edu/contact-us'}, {'type': 'alumni page', 'url': 'https://www.aiub.edu/Alumni'}, {'type': 'scholarships page', 'url': 'https://www.aiub.edu/admission/scholarships'}, {'type': 'faculty list', 'url': 'https://www.aiub.edu/Faculty-List'}, {'type': 'partnerships page', 'url': 'https://www.aiub.edu/partnerships/internationalization'}, {'type': 'tuition fee page', 'url': 'https://www.aiub.edu/tuition-fee'}, {'type': 'research page', 'url': 'https://www.aiub.edu/research'}, {'type': 'publications page', 'url': 'https://www.aiub.edu/publicatio

# Welcome to AIUB: Where Leaders Are Manufactured (With a Few Nuts and Bolts!)

## 🤖 About Us

Welcome to the **American International University-Bangladesh (AIUB)**! We're not just your average institution of higher learning - we're a **leader-making machine**! Our mission? To ensure our students graduate ready to face the world - armed with knowledge, skills, and perhaps an unusual fondness for coffee.

## 📚 Why Spend Your Time Here? 

- **Tech-Centric Teaching:** We embrace technology so much that our professors may as well have circuit boards instead of brains! (Just kidding... sort of).
- **Urban Green Campus:** Our campus is like an eco-friendly oasis, where you can find shade, inspiration, and maybe even a squirrel giving a motivational speech.
- **Endless Resources:** Libraries, faculty—dare we say, we have it ALL! Plus, stunning views while you cram for exams.

## 🎓 Programs and Courses

Choose from a dazzling array of programs (yes, we’re dazzling—you'll need your shades):

- **Arts & Social Sciences:** Because every superhero needs a secret identity!
- **Business Administration:** Where you learn how to make money and avoid existential crises.
- **Engineering:** Build bridges, robots, or just really complicated spreadsheets. The limit is your imagination (and possibly physics).
- **Health and Life Sciences:** Because we need skilled folks who know how to save the world - or at least bandage a paper cut.

## 🤝 Our Customers

Our customers? Let's face it, they aren't just customers; they are **future leaders, thinkers, and innovators!** We cater to anyone who dreams of changing the world—one lecture (or late-night study session) at a time.

## 💼 Careers at AIUB 

We also have jobs (yeah, you heard us!):
- **Sarcastic Librarian:** Unofficial title available upon frequent usage of "Shhhh"!
- **Innovative Professor:** Must love late nights, coffee, and explaining complex theories in three different metaphors.
- **Cultural Affairs Coordinator:** Your mission—make sure there are as many fun events as there are days in the week. (Warning: must love confetti).

## 🎉 Company Culture

At AIUB, team spirit runs high! We practice an open-door policy—with the occasional doorbell for dramatic effect. Whether it’s through studying, sports, or campus gatherings, we believe in **working hard and hardly working** (wait, what?). 

We’re family here—if you don’t eat pizza with us on Fridays, are you really even part of the crew?

## ✉️ Contact Us (Before We Run Out of Jokes)

Interested? Want to join our elite circle of future change-makers? Or just curious about how we make the best coffee on campus? 

Reach out! And remember to turn on your JavaScript (it’s a prerequisite for maximum fun) before you visit our website! Who knows—you might just discover your future here!

---

At AIUB, the future isn’t just bright; it’s downright **dazzling** (with a chance of awesome). Join us and let’s create tomorrow together—one syllabus at a time! ✨

# Minor Improvements

In [58]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": create_system_prompt},
            {
                "role": "user",
                "content": get_brochure_user_prompt(company_name, url),
            },
        ],
        stream=True,
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```", "").replace("markdown", "")
        update_display(
            Markdown(response), display_id=display_handle.display_id
        )

In [42]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face: The Cuddliest AI Community Around!

![Hugging Face Logo](https://huggingface.co/static/huggingface_logo_new.svg)

## 🌟 Who We Are

At Hugging Face, we're on a mission to **build the future of AI**—all while making sure our community has enough hugs (virtually, of course). Not only do we love to cuddle with data, but we also want to pack it into the most magical models you've ever seen!

## 🤖 What We Do

- **Models Galore**: With over 1 million models, call us the “Model UN” of AI, but with way less politics and way more hugging! 🌍🤗
- **Dazzling Datasets**: We boast a robust library of 250k+ datasets—it's basically an all-you-can-eat buffet for data enthusiasts. And yes, we have options for those on a gluten-free diet. 📊🥳
- **Spaces to Explore**: Dive into the creativity ocean! Generate applications faster than you can say "Hugging Face" with our über-cool Spaces. You're just a few clicks away from the next viral sensation! 🚀

## 🎉 Customers Who Give Us Warm Fuzzies

Join over **50,000 organizations** (including tech giants like Google, Microsoft, and Amazon) that are already leveraging our platform. From innovative start-ups to established enterprises, we’re the preferred partner for anyone looking to embrace the AI community. 🎈

## 💼 Careers & Culture at Hugging Face

At Hugging Face, we don't just work; we build friendships. Our team is powered by coffee, kindness, and a touch of AI magic! Join us if you:
1. Love collaboration and sharing ideas (and memes!).
2. Think coding is like a warm hug for the brain.
3. Want to be part of a community where every day feels like an exciting new adventure!

### Open Positions:
- AI Wizard
- Data Sorcerer
- Hugging Specialist (kidding! But please bring your warmth).

If you're dreaming of a career where you can play with language models, create amazing applications, and share hugs (virtually, of course), **we want you!** Check out our [job listings](https://huggingface.co/jobs)!

## 🌐 Join the Hugging Community

Whether you're a seasoned pro in machine learning or a curious newcomer, we welcome everyone! Come aboard, and together we’ll create and explore AI innovations. 🥳🎊

Ready to join the hug-fest? Sign up now at [Hugging Face](https://huggingface.co) and be part of the **AI future** that's warm, welcoming, and lots of fun!

> Remember, friendships can overshadow any algorithm. So let’s calculate those cuddles! 🤗

---

For more information, help, or just a chat, follow us on [GitHub](https://github.com/huggingface) or [Twitter](https://twitter.com/huggingface). Let’s connect, collaborate, and revolutionize AI together!

**Hugging Face** – The AI Community Building the Future (one hug at a time)!

# 3rd Call

In [63]:
system_prompt = "You are an expert English-to-Bangla translator. \
You accurately and professionally translate any given text into Bangla, preserving the original format, tone, \
and context without any loss of meaning. Respond in Markdown."

In [60]:
def get_translated_brochure_user_prompt(brochure_text):
    result = "Below is a brochure written in Markdown format:\n\n"
    result += f"{brochure_text}\n\n"
    result += "Please translate this brochure into Bangla, preserving the original format, tone, and context accurately."
    return result


In [ ]:
def translate_brochure(company_name, url):
    content = create_brochure(company_name, url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": get_translated_brochure_user_prompt(content),
            },
        ],
    )
    result = response.choices[0].message.content
    return result

In [78]:
display(translate_brochure("HuggingFace", "https://huggingface.co"))

Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face ব্রোশিওর

## আমাদের সম্পর্কে
**Hugging Face** কৃত্রিম বুদ্ধিমত্তার (AI) সম্প্রদায়ের অগ্রভাগে, মেশিন লার্নিংয়ের ভবিষ্যত নির্মাণে নিবেদিত। আমরা একটি শক্তিশালী প্ল্যাটফর্ম প্রদান করি যেখানে ব্যক্তি এবং সংস্থাগুলি **মডেল**, **ডেটাসেট** এবং অ্যাপ্লিকেশনগুলিতে সহযোগিতা করতে পারে, যা কৃত্রিম বুদ্ধিমত্তার ক্ষেত্রে উদ্ভাবন এবং বৃদ্ধি সহজ করে তোলে।

## আমাদের অফারিংস
- **মডেল**: টেক্সট থেকে ইমেজ, অডিও এবং 3D পর্যন্ত **1 মিলিয়নেরও বেশি মডেল** অ্যাক্সেস এবং শেয়ার করুন।
- **ডেটাসেট**: বিভিন্ন এমএল কাজের জন্য তৈরিকৃত **250ক ডেটাসেট** অন্বেষণ এবং ব্যবহার করুন।
- **স্পেস**: সহজেই অ্যাপ্লিকেশন তৈরি এবং চালান, বিভিন্ন ডোমেনের জন্য কাস্টমাইজড আউটপুট উৎপাদনের বিকল্পসহ।

### মূল হাইলাইটস
- **ট্রেন্ডিং মডেল**: Hugging Face-এ সর্বশেষ মডেলগুলি অন্তর্ভুক্ত:
  - `agentica-org/DeepCoder-14B-Preview` 
  - `meta-llama/Llama-4-Scout-17B-16E-Instruct`
- **সহযোগিতা**: Google, Amazon, Microsoft-এর মতো প্রধান উদ্যোগসমূহসহ **50,000-এরও বেশি সংস্থার** সঙ্গে একত্রে কাজ করুন।

## কোম্পানির সংস্কৃতি
Hugging Face-এ, আমরা **সম্প্রদায়** এবং **সহযোগিতার** শক্তিতে বিশ্বাস করি। আমরা একটি অন্তর্ভুক্তিমূলক পরিবেশ তৈরি করতে চেষ্টা করি যেখানে উদ্ভাবন মুক্ত সফটওয়্যার নীতিগুলির মাধ্যমে প্রসারিত হয়। আমাদের সংস্কৃতির বৈশিষ্ট্যগুলি হল:

- **স্বচ্ছতা**: খোলামেলা যোগাযোগ এবং জ্ঞানের শেয়ারিং।
- **সম্প্রদায়ের বিনিয়োগ**: ব্যবহারকারীদের এবং সংস্থাগুলির সক্রিয় অংশগ্রহণ এবং অবদান।
- **সহায়তা**: প্রতিটি স্তরের ব্যবহারকারীদের জন্য নিবেদিত সহায়তা প্রদান করা, নিশ্চিত করে যে সবাই AI এর সুবিধা নিতে পারে।

## Hugging Face-এ ক্যারিয়ার
আমরা সদা কর্মঠ ব্যক্তিদের খোঁজে থাকি যারা আমাদের টিমে যোগদান করতে ইচ্ছুক। **AI-এর গণতান্ত্রিকীকরণ ও মেশিন লার্নিংয়ের ভবিষ্যত রূপান্তরে আমাদের মিশনে** আমাদের সাথে যোগ দিন। বর্তমান ক্যারিয়ারের সুযোগগুলি বিভিন্ন ক্ষেত্রে বিস্তৃত, যেমন:

- **সফটওয়্যার ডেভেলপমেন্ট**
- **ডেটা সায়েন্স**
- **সামাজিক সম্পকর্কার্যক্রম**
- **মার্কেটিং**

বর্তমান চাকরির সুযোগ এবং কিভাবে আবেদন করতে হয় সে সম্পর্কে আরও জানার জন্য আমাদের ক্যারিয়ার পৃষ্ঠা দেখুন।

## যোগাযোগ করুন
Hugging Face এর সাথে সম্ভাবনাগুলি আবিষ্কার করুন! আমাদের সেবা অন্বেষণ করতে, আমাদের সম্প্রদায় সম্পর্কে আরও জানতে, বা অংশীদারত্বের সুযোগের জন্য আমাদের সাথে যোগাযোগ করতে [huggingface.co](https://huggingface.co) তে যান।

---

একসঙ্গে, আমরা AI-এর ভবিষ্যত নির্মাণ করি!

None